<a href="https://colab.research.google.com/github/svedison/JDRE-Research/blob/main/Benchmarks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from scipy.stats import spearmanr
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 1. BIOSSES Dataset (example sentences) - tend to be used for contrastive learning scenarios
# Each tuple: (sentence1, sentence2, similarity_score)
biosses_data = [
    ("Sunitinib is a tyrosine kinase inhibitor.", "The patient was given sunitinib for cancer.", 4.5),
    ("Insulin regulates glucose in the body.", "The patient was started on insulin for diabetes.", 4.0),
    ("Warfarin is an anticoagulant.", "He was prescribed warfarin due to high clot risk.", 4.8),
    ("Metformin lowers blood sugar.", "She takes metformin for type 2 diabetes.", 4.7),
    ("Atorvastatin reduces cholesterol levels.", "The patient is on atorvastatin to manage cholesterol.", 4.6),
    ("Amoxicillin treats bacterial infections.", "Amoxicillin was prescribed for an ear infection.", 4.9),
    ("Lisinopril is used for hypertension.", "He takes lisinopril to control his high blood pressure.", 4.3),
    ("Levothyroxine replaces thyroid hormone.", "She is on levothyroxine due to hypothyroidism.", 4.4),
    ("Albuterol is a bronchodilator.", "The patient uses albuterol to relieve asthma symptoms.", 4.6),
    ("Omeprazole reduces stomach acid.", "Omeprazole was given for acid reflux management.", 4.5),
]

sentences1 = [pair[0] for pair in biosses_data]
sentences2 = [pair[1] for pair in biosses_data]
gold_scores = [pair[2] for pair in biosses_data]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. SentenceTransformer biomedical model
# Use a publicly accessible BioBERT model fine-tuned on SNLI, MNLI, STS tasks
model_name = "pritamdeka/BioBERT-snli-mnli-stsb"

try:
    bio_sentvec_repl = SentenceTransformer(model_name)
    print(f"Loaded model: {model_name}")
except Exception as e:
    print(f"Failed to load model '{model_name}': {e}")
    # Fallback to a general-purpose model
    fallback_model_name = "sentence-transformers/all-mpnet-base-v2"
    print(f"Loading fallback model: {fallback_model_name}")
    bio_sentvec_repl = SentenceTransformer(fallback_model_name)


# 3. PubMedBERT embeddings
print("Loading PubMedBERT...")
pubmedbert_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
pubmedbert_tokenizer = AutoTokenizer.from_pretrained(pubmedbert_name)
pubmedbert_model = AutoModel.from_pretrained(pubmedbert_name).to(device)
pubmedbert_model.eval()

def get_pubmedbert_embedding(sentences):
    inputs = pubmedbert_tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = pubmedbert_model(**inputs)
        cls_embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token
    return cls_embeddings.cpu().numpy()

# 4. CLIP-style dual encoder model
class CLIPStyleModel(torch.nn.Module):
    def __init__(self, bio_model_name, clinical_model_name, proj_dim=256, max_length=128):
        super().__init__()
        self.bio_tokenizer = AutoTokenizer.from_pretrained(bio_model_name)
        self.clinical_tokenizer = AutoTokenizer.from_pretrained(clinical_model_name)
        self.bio_encoder = AutoModel.from_pretrained(bio_model_name).to(device)
        self.clinical_encoder = AutoModel.from_pretrained(clinical_model_name).to(device)
        hidden_size = self.bio_encoder.config.hidden_size
        self.max_length = max_length
        self.bio_proj = torch.nn.Sequential(
            torch.nn.Linear(hidden_size, proj_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(proj_dim, proj_dim)
        )
        self.clinical_proj = torch.nn.Sequential(
            torch.nn.Linear(hidden_size, proj_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(proj_dim, proj_dim)
        )

    def encode(self, texts, tokenizer, encoder):
        inputs = tokenizer(texts, padding=True, truncation=True, max_length=self.max_length, return_tensors="pt").to(device)
        outputs = encoder(**inputs)
        cls_token = outputs.last_hidden_state[:, 0, :]
        return cls_token

    def get_embeddings(self, bio_texts, clinical_texts):
        bio_cls = self.encode(bio_texts, self.bio_tokenizer, self.bio_encoder)
        clinical_cls = self.encode(clinical_texts, self.clinical_tokenizer, self.clinical_encoder)
        bio_emb = F.normalize(self.bio_proj(bio_cls), dim=1)
        clinical_emb = F.normalize(self.clinical_proj(clinical_cls), dim=1)
        return bio_emb.cpu().detach().numpy(), clinical_emb.cpu().detach().numpy()

# Instantiate CLIP Model
clip_model = CLIPStyleModel(
    bio_model_name="michiyasunaga/BioLinkBERT-base",
    clinical_model_name="emilyalsentzer/Bio_ClinicalBERT",
    proj_dim=256,
    max_length=128
)

clip_model.eval()

# 5. Compute cosine similarity & Spearman correlation

def evaluate_model(model_name, encode_fn1, encode_fn2):
    emb1 = encode_fn1(sentences1)
    emb2 = encode_fn2(sentences2)
    sims = [cosine_similarity([e1], [e2])[0][0] for e1, e2 in zip(emb1, emb2)]
    corr, _ = spearmanr(sims, gold_scores)
    print(f"{model_name} Spearman Correlation: {corr:.4f}")
    return corr

print("\nEvaluating models on BIOSSES...")

# BioSentVec replacement
def encode_biosentvec(sentences):
    return bio_sentvec_repl.encode(sentences, convert_to_numpy=True)

# PubMedBERT
def encode_pubmedbert(sentences):
    return get_pubmedbert_embedding(sentences)

# CLIP model embeddings
def encode_clip_bio(sentences):
    # Here for BIOSSES we assume both sentences are biomedical domain,
    # so encode both sides with bio_encoder+proj
    inputs = clip_model.bio_tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = clip_model.bio_encoder(**inputs)
        cls_token = outputs.last_hidden_state[:, 0, :]
        emb = F.normalize(clip_model.bio_proj(cls_token), dim=1)
    return emb.cpu().numpy()

def encode_clip_clinical(sentences):
    inputs = clip_model.clinical_tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = clip_model.clinical_encoder(**inputs)
        cls_token = outputs.last_hidden_state[:, 0, :]
        emb = F.normalize(clip_model.clinical_proj(cls_token), dim=1)
    return emb.cpu().numpy()

# For BIOSSES pairs: encode first sentence with bio encoder, second with clinical encoder
def encode_clip_pair(sentences1, sentences2):
    emb1 = encode_clip_bio(sentences1)
    emb2 = encode_clip_clinical(sentences2)
    return emb1, emb2

emb_clip_1, emb_clip_2 = encode_clip_pair(sentences1, sentences2)
clip_sims = [cosine_similarity([e1], [e2])[0][0] for e1, e2 in zip(emb_clip_1, emb_clip_2)]
clip_corr, _ = spearmanr(clip_sims, gold_scores)
print(f"CLIP-style Model Spearman Correlation: {clip_corr:.4f}")

# BioSentVec replacement
evaluate_model("BioSentVec replacement", encode_biosentvec, encode_biosentvec)

# PubMedBERT
evaluate_model("PubMedBERT", encode_pubmedbert, encode_pubmedbert)

print("\n--- Done ---\n")

# 6. BioASQ retrieval evaluation below

Failed to load model 'pritamdeka/BioBERT-snli-mnli-stsb': pritamdeka/BioBERT-snli-mnli-stsb is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`
Loading fallback model: sentence-transformers/all-mpnet-base-v2
Loading PubMedBERT...

Evaluating models on BIOSSES...
CLIP-style Model Spearman Correlation: -0.0427
BioSentVec replacement Spearman Correlation: 0.4451
PubMedBERT Spearman Correlation: 0.5976

--- Done ---



In [2]:
from sentence_transformers import SentenceTransformer

# Use a publicly accessible BioBERT model fine-tuned on SNLI, MNLI, STS tasks
model_name = "pritamdeka/BioBERT-snli-mnli-stsb"

try:
    model = SentenceTransformer(model_name)
    print(f"Loaded model: {model_name}")
except Exception as e:
    print(f"Failed to load model '{model_name}': {e}")
    # Fallback to a general-purpose model or raise error
    fallback_model_name = "sentence-transformers/all-mpnet-base-v2"
    print(f"Loading fallback model: {fallback_model_name}")
    model = SentenceTransformer(fallback_model_name)

# Example usage
sentences = [
    "Sunitinib is a tyrosine kinase inhibitor.",
    "The patient was given sunitinib for cancer."
]

embeddings = model.encode(sentences)
print("Embeddings shape:", embeddings.shape)

Failed to load model 'pritamdeka/BioBERT-snli-mnli-stsb': pritamdeka/BioBERT-snli-mnli-stsb is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`
Loading fallback model: sentence-transformers/all-mpnet-base-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings shape: (2, 768)


In [6]:
#BioASQ retrieval
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Example BioASQ-style dataset loader
# For demonstration, a tiny mock BioASQ retrieval dataset:
bioasq_data = [
    {
        "query": "What drugs are used to treat diabetes?",
        "documents": [
            "Insulin regulates glucose in the body.",
            "Warfarin is an anticoagulant.",
            "Metformin lowers blood sugar.",
            "Amoxicillin treats bacterial infections."
        ],
        "relevant_doc_indices": [0, 2]  # indices of relevant docs for this query
    },
    {
        "query": "What medications treat high cholesterol?",
        "documents": [
            "Atorvastatin reduces cholesterol levels.",
            "Lisinopril is used for hypertension.",
            "Omeprazole reduces stomach acid.",
            "Levothyroxine replaces thyroid hormone."
        ],
        "relevant_doc_indices": [0]
    }
]

# Retrieval evaluation metrics

def mean_reciprocal_rank(ranked_indices, relevant_indices):
    for rank, idx in enumerate(ranked_indices, start=1):
        if idx in relevant_indices:
            return 1.0 / rank
    return 0.0

def recall_at_k(ranked_indices, relevant_indices, k=5):
    return len(set(ranked_indices[:k]) & set(relevant_indices)) / len(relevant_indices)

# Encode functions from previous example (reuse them)

# For simplicity: encode a single query or document list using same encoding functions

# Assume encode_biosentvec, encode_pubmedbert, encode_clip_bio, encode_clip_clinical are defined (from previous snippet)

# We'll encode query and docs for each model as follows:

def encode_clip_query_and_docs(query, docs):
    # Query with bio encoder, docs with clinical encoder
    q_emb = encode_clip_bio([query])
    d_embs = encode_clip_clinical(docs)
    return q_emb, d_embs

def encode_biosentvec_query_and_docs(query, docs):
    q_emb = bio_sentvec_repl.encode([query], convert_to_numpy=True)
    d_embs = bio_sentvec_repl.encode(docs, convert_to_numpy=True)
    return q_emb, d_embs

def encode_pubmedbert_query_and_docs(query, docs):
    q_emb = get_pubmedbert_embedding([query])
    d_embs = get_pubmedbert_embedding(docs)
    return q_emb, d_embs

# Run retrieval evaluation

def evaluate_retrieval(data, encode_fn, model_name, k=5):
    mrrs = []
    recalls = []
    for item in data:
        query = item["query"]
        docs = item["documents"]
        relevant = item["relevant_doc_indices"]

        q_emb, d_embs = encode_fn(query, docs)
        sims = cosine_similarity(q_emb, d_embs)[0]  # similarity scores of docs to query
        ranked_indices = np.argsort(sims)[::-1]  # descending order

        mrr = mean_reciprocal_rank(ranked_indices, relevant)
        rec = recall_at_k(ranked_indices, relevant, k=k)

        mrrs.append(mrr)
        recalls.append(rec)

    print(f"{model_name} Retrieval Results:")
    print(f"  Mean Reciprocal Rank (MRR): {np.mean(mrrs):.4f}")
    print(f"  Recall@{k}: {np.mean(recalls):.4f}")
    print()

# Evaluate all 3 models on BioASQ mock

print("Running BioASQ retrieval benchmark:\n")

evaluate_retrieval(bioasq_data, encode_biosentvec_query_and_docs, "BioSentVec replacement")
evaluate_retrieval(bioasq_data, encode_pubmedbert_query_and_docs, "PubMedBERT")
evaluate_retrieval(bioasq_data, encode_clip_query_and_docs, "CLIP-style Model")

print("Retrieval complete.")

Running BioASQ retrieval benchmark...

BioSentVec replacement Retrieval Results:
  Mean Reciprocal Rank (MRR): 1.0000
  Recall@5: 1.0000

PubMedBERT Retrieval Results:
  Mean Reciprocal Rank (MRR): 0.7500
  Recall@5: 1.0000

CLIP-style Model Retrieval Results:
  Mean Reciprocal Rank (MRR): 0.5000
  Recall@5: 1.0000

BioASQ retrieval evaluation complete.
